# Recon 3D CHO GPRs

### 1. Generation of Orthologs List for Human and CHO genes with NIH gene IDs
A dataset containing Ensemb IDs of ortholog genes between Human and Chinese Hamster was obtained from BioMart. Two  datasets containing a complete gene lists for the entire Human and Chinese Hamster organisms were obtained from NIH Datasets https://www.ncbi.nlm.nih.gov/datasets/. Furthermore, two datasets containing Ensembl and Entrez IDs from both, Human and CHO, were obtained from DAVID https://david.ncifcrf.gov/tools.jsp. The final "orthologs" dataset generated contains all the ortholog genes from Human and Hamster with its corresponding NIH IDs. Finally, one dataset containing CHO and Human gene orthologs from Paranoid database https://inparanoid.sbc.su.se was used to fill empty genes.

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

### Finding CHO orthologs for Human GPRs in Recon3D
Using the "orthologs" dataset generated above, we can map all the CHO genes in the recon 3D dataset GPR column and generate a new columns "CHO GPR"

In [9]:
#Generate recon3d df
df1 = pd.read_excel('../Data/GPR_Curation/recon3d_gprs.xlsx')
df1

,m_reaction,m_gene_reaction_rule,m_metabolites,m_subsystem,m_gene,seq_uniprot,seq_len,seq_file,seq_num_structures,seq_num_experimental_structures,struct_id,struct_chain_id,struct_chain_seq_coverage,struct_is_experimental,struct_pdb,struct_resolution,struct_chemicals,struct_file
0,11DOCRTSLte,5243.1,h2o[c];atp[c];h[c];11docrtsl[c];11docrtsl[e];p...,"Transport, extracellular",5243.1,P08183-1,1280.0,P08183-1.fasta,21.0,21.0,3g61-A,A,80.5,1.0,3g61,4.35,2J8,3g61-A_clean.pdb
1,11DOCRTSTRNte,5243.1,atp[c];pi[c];11docrtstrn[c];h[c];11docrtstrn[e...,"Transport, extracellular",5243.1,P08183-1,1280.0,P08183-1.fasta,21.0,21.0,3g61-A,A,80.5,1.0,3g61,4.35,2J8,3g61-A_clean.pdb
2,12DHCHOLabc,8714.3 or 8647.1 or 8714.2 or 8714.1 or 1244.1...,atp[c];12dhchol[e];12dhchol[c];pi[c];h[c];h2o[...,"Transport, extracellular",1244.1,Q92887-1,1545.0,Q92887-1.fasta,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12DHCHOLabc,8714.3 or 8647.1 or 8714.2 or 8714.1 or 1244.1...,atp[c];12dhchol[e];12dhchol[c];pi[c];h[c];h2o[...,"Transport, extracellular",8647.1,O95342-1,1321.0,O95342-1.fasta,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12DHCHOLabc,8714.3 or 8647.1 or 8714.2 or 8714.1 or 1244.1...,atp[c];12dhchol[e];12dhchol[c];pi[c];h[c];h2o[...,"Transport, extracellular",10257.1,O15439-1,1325.0,O15439-1.fasta,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22107,r2525,8501.1,gln_L[c];gln_L[e],"Transport, extracellular",8501.1,O75387-1,559.0,O75387-1.fasta,2.0,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb
22108,r2526,8501.1,ser_L[e];ser_L[c],"Transport, extracellular",8501.1,O75387-1,559.0,O75387-1.fasta,2.0,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb
22109,r2532,8501.1,asn_L[c];asn_L[e],"Transport, extracellular",8501.1,O75387-1,559.0,O75387-1.fasta,2.0,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb
22110,r2534,8501.1,thr_L[c];thr_L[e],"Transport, extracellular",8501.1,O75387-1,559.0,O75387-1.fasta,2.0,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb


In [10]:
import cobra
from cobra.io.mat import load_matlab_model

recon3d_model = load_matlab_model('../Data/GPR_Curation/Recon3D_301.mat')
recon3d_model

/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
No defined compartments in model Recon3D. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


Name,Recon3D
Memory address,286c4b460
Number of metabolites,8399
Number of reactions,13543
Number of genes,3697
Number of groups,111
Objective expression,1.0*biomass_reaction - 1.0*biomass_reaction_reverse_32a6c
Compartments,"c, l, m, r, e, x, n, g, i"


In [11]:
attributes = []
for reaction in tqdm(recon3d_model.reactions):
    attributes.append([reaction.id, reaction.name, reaction.reaction, reaction.gpr, 
                       reaction.subsystem, reaction.lower_bound, reaction.upper_bound])

df2 = pd.DataFrame(data=attributes, columns=['m_reaction', 'Reaction Name', 'm_metabolites', 'm_gene_reaction_rule', 'm_subsystem', 'Lower bound', 'Upper bound'])
df2['m_reaction'] = df2['m_reaction'].str.replace("[", "_")
df2['m_reaction'] = df2['m_reaction'].str.replace("]", "")
df2

  0%|          | 0/13543 [00:00<?, ?it/s]

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_39284/3575368116.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['m_reaction'] = df2['m_reaction'].str.replace("[", "_")
/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_39284/3575368116.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['m_reaction'] = df2['m_reaction'].str.replace("]", "")


,m_reaction,Reaction Name,m_metabolites,m_gene_reaction_rule,m_subsystem,Lower bound,Upper bound
0,10FTHF5GLUtl,"5-Glutamyl-10Fthf Transport, Lysosomal",10fthf5glu[c] --> 10fthf5glu[l],,"Transport, lysosomal",0.0,1000.0
1,10FTHF5GLUtm,"5-Glutamyl-10Fthf Transport, Mitochondrial",10fthf5glu[m] --> 10fthf5glu[c],,"Transport, mitochondrial",0.0,1000.0
2,10FTHF6GLUtl,"6-Glutamyl-10Fthf Transport, Lysosomal",10fthf6glu[c] --> 10fthf6glu[l],,"Transport, lysosomal",0.0,1000.0
3,10FTHF6GLUtm,"6-Glutamyl-10Fthf Transport, Mitochondrial",10fthf6glu[m] --> 10fthf6glu[c],,"Transport, mitochondrial",0.0,1000.0
4,10FTHF7GLUtl,"7-Glutamyl-10Fthf Transport, Lysosomal",10fthf7glu[c] --> 10fthf7glu[l],,"Transport, lysosomal",0.0,1000.0
...,...,...,...,...,...,...,...
13538,CYOR_u10mi,CYOR_u10mi,2.0 ficytC[m] + 2.0 h[m] + q10h2[m] --> 2.0 fo...,(7384.1 and 7388.1 and 4519.1 and 29796.2 and ...,Oxidative phosphorylation,0.0,1000.0
13539,Htmi,Htmi,h[i] --> h[m],9016.1 or 7352.2 or 7352.1 or 7351.1 or 9016.2...,"Transport, mitochondrial",0.0,1000.0
13540,NADH2_u10mi,NADH2_u10mi,5.0 h[m] + nadh[m] + q10[m] --> 4.0 h[i] + nad...,(4715.1 and 4720.1 and 4719.1 and 4700.1 and 5...,Oxidative phosphorylation,0.0,1000.0
13541,CYOOm3i,CYOOm3i,4.0 focytC[m] + 7.92 h[m] + o2[m] --> 4.0 ficy...,1351.1 and 1347.1 and 1329.1 and 1327.1 and 34...,Oxidative phosphorylation,0.0,1000.0


In [12]:
recon3d = pd.concat([df2, df1])
recon3d = recon3d.groupby('m_reaction').first()
recon3d = recon3d.reset_index(drop = False)
recon3d['m_gene_reaction_rule'] = recon3d['m_gene_reaction_rule'].replace(np.nan,'',regex=True)
recon3d

,m_reaction,Reaction Name,m_metabolites,m_gene_reaction_rule,m_subsystem,Lower bound,Upper bound,m_gene,seq_uniprot,seq_len,...,seq_num_structures,seq_num_experimental_structures,struct_id,struct_chain_id,struct_chain_seq_coverage,struct_is_experimental,struct_pdb,struct_resolution,struct_chemicals,struct_file
0,10FTHF5GLUtl,"5-Glutamyl-10Fthf Transport, Lysosomal",10fthf5glu[c] --> 10fthf5glu[l],,"Transport, lysosomal",0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None
1,10FTHF5GLUtm,"5-Glutamyl-10Fthf Transport, Mitochondrial",10fthf5glu[m] --> 10fthf5glu[c],,"Transport, mitochondrial",0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None
2,10FTHF6GLUtl,"6-Glutamyl-10Fthf Transport, Lysosomal",10fthf6glu[c] --> 10fthf6glu[l],,"Transport, lysosomal",0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None
3,10FTHF6GLUtm,"6-Glutamyl-10Fthf Transport, Mitochondrial",10fthf6glu[m] --> 10fthf6glu[c],,"Transport, mitochondrial",0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None
4,10FTHF7GLUtl,"7-Glutamyl-10Fthf Transport, Lysosomal",10fthf7glu[c] --> 10fthf7glu[l],,"Transport, lysosomal",0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13538,steroids,Steroids (from HMR),46dhoxquin[c] + 48dhoxquin[c] + C05299[c] + C0...,,Exchange/demand reaction,0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None
13539,vitaminA,Vitamina (from HMR),CE1761[c] + CE1761[r] + CE2954[c] + CE2954[r] ...,,Exchange/demand reaction,0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None
13540,vitaminD,Vitamind (from HMR),CE2204[c] + CE2204[m] + CE2207[c] + CE2207[m] ...,,Exchange/demand reaction,0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None
13541,vitaminE,Vitamine (from HMR),CE1925[c] + CE5854[c] --> M03143[c],,Exchange/demand reaction,0.0,1000.0,NaN,None,NaN,...,NaN,NaN,None,None,NaN,NaN,None,NaN,None,None


In [13]:
# Orthologs dict
orthologs_dict = orthologs.set_index('Human GeneID')['CHO GeneID'].to_dict()
orthologs_dict

{'4535': '3979183',
 '4536': '3979184',
 '4512': '3979185',
 '4513': '3979186',
 '4509': '3979187',
 '4508': '3979188',
 '4514': '3979189',
 '4537': '3979190',
 '4539': '3979178',
 '4538': '3979179',
 '4540': '3979180',
 '4541': '3979181',
 '4519': '3979182',
 '54859': '100764169',
 '203523': '100764457',
 '55813': '100761515',
 '901': '100753075',
 '9840': '100754149',
 '6919': '100768573',
 '5885': '100750544',
 '1910': '100762726',
 '26022': '100762391',
 '84432': '100768325',
 '90187': '100771706',
 '1763': '100761842',
 '4026': '100753874',
 '79184': '100753541',
 '64795': '100766802',
 '100507003': '100772084',
 '4841': '100757453',
 '84311': '100755070',
 '10864': '100757046',
 '6857': '100753843',
 '51268': '100765858',
 '59272': '100772552',
 '58516': '100760112',
 '645104': '100769210',
 '64857': '100761461',
 '165904': '100756551',
 '8722': '100756958',
 '1896': '100768945',
 '28959': '100764547',
 '125115': '100758909',
 '2188': '103161842',
 '144809': '113832672',
 '10873'

In [14]:
# Extract GPR info from Recon3D and swap gene IDs from human to CHO
import re
cho_gpr = []

for row in recon3d['m_gene_reaction_rule']:
    row = str(row)
    if row != "":
        gpr = re.findall('[\d.]*\d+', row)
        new_gpr = row
        for g in gpr:
            human_g = g.split('.')[0]
            try:
                cho_g = orthologs_dict[human_g]
                if cho_g == '':
                    cho_g = f'h{human_g}'
            except:
                cho_g = f'h{human_g}'
            new_gpr = new_gpr.replace(g, cho_g)
    elif row == '':
        new_gpr = ''
        
    cho_gpr.append(new_gpr)

In [15]:
# Generate Recon3D dataset with the addition of GPR with CHO genes
recon3d['CHO GPR'] = cho_gpr
recon3d.to_excel('../Data/GPR_Curation/recon3D_chogprs.xlsx')
recon3d

,m_reaction,Reaction Name,m_metabolites,m_gene_reaction_rule,m_subsystem,Lower bound,Upper bound,m_gene,seq_uniprot,seq_len,...,seq_num_experimental_structures,struct_id,struct_chain_id,struct_chain_seq_coverage,struct_is_experimental,struct_pdb,struct_resolution,struct_chemicals,struct_file,CHO GPR
0,10FTHF5GLUtl,"5-Glutamyl-10Fthf Transport, Lysosomal",10fthf5glu[c] --> 10fthf5glu[l],,"Transport, lysosomal",0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,
1,10FTHF5GLUtm,"5-Glutamyl-10Fthf Transport, Mitochondrial",10fthf5glu[m] --> 10fthf5glu[c],,"Transport, mitochondrial",0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,
2,10FTHF6GLUtl,"6-Glutamyl-10Fthf Transport, Lysosomal",10fthf6glu[c] --> 10fthf6glu[l],,"Transport, lysosomal",0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,
3,10FTHF6GLUtm,"6-Glutamyl-10Fthf Transport, Mitochondrial",10fthf6glu[m] --> 10fthf6glu[c],,"Transport, mitochondrial",0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,
4,10FTHF7GLUtl,"7-Glutamyl-10Fthf Transport, Lysosomal",10fthf7glu[c] --> 10fthf7glu[l],,"Transport, lysosomal",0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13538,steroids,Steroids (from HMR),46dhoxquin[c] + 48dhoxquin[c] + C05299[c] + C0...,,Exchange/demand reaction,0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,
13539,vitaminA,Vitamina (from HMR),CE1761[c] + CE1761[r] + CE2954[c] + CE2954[r] ...,,Exchange/demand reaction,0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,
13540,vitaminD,Vitamind (from HMR),CE2204[c] + CE2204[m] + CE2207[c] + CE2207[m] ...,,Exchange/demand reaction,0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,
13541,vitaminE,Vitamine (from HMR),CE1925[c] + CE5854[c] --> M03143[c],,Exchange/demand reaction,0.0,1000.0,NaN,None,NaN,...,NaN,None,None,NaN,NaN,None,NaN,None,None,


In [21]:
# Load Recon 3D dataset
recon3d = pd.read_excel('../Data/GPR_Curation/recon3D_chogprs.xlsx', index_col=0)
recon3d

,m_reaction,Reaction Name,m_metabolites,m_gene_reaction_rule,m_subsystem,Lower bound,Upper bound,m_gene,seq_uniprot,seq_len,...,seq_num_experimental_structures,struct_id,struct_chain_id,struct_chain_seq_coverage,struct_is_experimental,struct_pdb,struct_resolution,struct_chemicals,struct_file,CHO GPR
0,10FTHF5GLUtl,"5-Glutamyl-10Fthf Transport, Lysosomal",10fthf5glu[c] --> 10fthf5glu[l],NaN,"Transport, lysosomal",0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10FTHF5GLUtm,"5-Glutamyl-10Fthf Transport, Mitochondrial",10fthf5glu[m] --> 10fthf5glu[c],NaN,"Transport, mitochondrial",0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10FTHF6GLUtl,"6-Glutamyl-10Fthf Transport, Lysosomal",10fthf6glu[c] --> 10fthf6glu[l],NaN,"Transport, lysosomal",0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10FTHF6GLUtm,"6-Glutamyl-10Fthf Transport, Mitochondrial",10fthf6glu[m] --> 10fthf6glu[c],NaN,"Transport, mitochondrial",0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10FTHF7GLUtl,"7-Glutamyl-10Fthf Transport, Lysosomal",10fthf7glu[c] --> 10fthf7glu[l],NaN,"Transport, lysosomal",0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13538,steroids,Steroids (from HMR),46dhoxquin[c] + 48dhoxquin[c] + C05299[c] + C0...,NaN,Exchange/demand reaction,0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13539,vitaminA,Vitamina (from HMR),CE1761[c] + CE1761[r] + CE2954[c] + CE2954[r] ...,NaN,Exchange/demand reaction,0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13540,vitaminD,Vitamind (from HMR),CE2204[c] + CE2204[m] + CE2207[c] + CE2207[m] ...,NaN,Exchange/demand reaction,0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13541,vitaminE,Vitamine (from HMR),CE1925[c] + CE5854[c] --> M03143[c],NaN,Exchange/demand reaction,0,1000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Subset of Recon 3D dataset with information on CHO GPRs
recon3d_cho = recon3d[recon3d['CHO GPR'].notnull()]
recon3d_cho = recon3d_cho.reset_index(drop=True)

In [23]:
recon3d_cho['m_metabolites'] = recon3d_cho['m_metabolites'].str.replace("[", "_")
recon3d_cho['m_metabolites'] = recon3d_cho['m_metabolites'].str.replace("]", "")
recon3d_cho

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_29406/1479258575.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  recon3d_cho['m_metabolites'] = recon3d_cho['m_metabolites'].str.replace("[", "_")
/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_29406/1479258575.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  recon3d_cho['m_metabolites'] = recon3d_cho['m_metabolites'].str.replace("]", "")


,m_reaction,Reaction Name,m_metabolites,m_gene_reaction_rule,m_subsystem,Lower bound,Upper bound,m_gene,seq_uniprot,seq_len,...,seq_num_experimental_structures,struct_id,struct_chain_id,struct_chain_seq_coverage,struct_is_experimental,struct_pdb,struct_resolution,struct_chemicals,struct_file,CHO GPR
0,11DOCRTSLte,"Transport of 11-Deoxycortisol, Extracellular",11docrtsl_c + atp_c + h2o_c --> 11docrtsl_e + ...,5243.1,"Transport, extracellular",0,1000,5243.1,P08183-1,1280.0,...,21.0,3g61-A,A,80.5,1.0,3g61,4.35,2J8,3g61-A_clean.pdb,100682536
1,11DOCRTSTRNte,"Transport of 11-Deoxycorticosterone, Extracell...",11docrtstrn_c + atp_c + h2o_c --> 11docrtstrn_...,5243.1,"Transport, extracellular",0,1000,5243.1,P08183-1,1280.0,...,21.0,3g61-A,A,80.5,1.0,3g61,4.35,2J8,3g61-A_clean.pdb,100682536
2,12DHCHOLabc,12-Dehydrocholic acid ABC bile acid transporter,12dhchol_c + atp_c + h2o_c --> 12dhchol_e + ad...,8714.3 or 8647.1 or 8714.2 or 8714.1 or 1244.1...,"Transport, extracellular",0,1000,1244.1,Q92887-1,1545.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100767558 or 100762480 or 100767558 or 1007675...
3,12DHCHOLt,12-Dehydrocholic acid transport via bicarbonat...,12dhchol_e + hco3_c <=> 12dhchol_c + hco3_e,28234.1 or 10599.1 or 123264.1 or 200931.1,"Transport, extracellular",-1000,1000,123264.1,Q9NPD5-1,702.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100774298 or 100774298 or 100751085 or 100753765
4,12DHCHOLt2,12-Dehydrocholic acid transport via sodium cot...,12dhchol_e + 2.0 na1_e --> 12dhchol_c + 2.0 na1_c,6554.1 or 6555.1,"Transport, extracellular",0,1000,6555.1,Q12908-1,348.0,...,0.0,H19894-X,X,99.7,0.0,NaN,NaN,NaN,NP_000443.1_model1_clean-X_clean.pdb,100756998 or 100689396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7672,r2525,Major Facilitator (Mfs) Tcdb:2.A.1.44.1,gln_L_e <=> gln_L_c,8501.1,"Transport, extracellular",-1000,1000,8501.1,O75387-1,559.0,...,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb,100757617
7673,r2526,Major Facilitator (Mfs) Tcdb:2.A.1.44.1,ser_L_e <=> ser_L_c,8501.1,"Transport, extracellular",-1000,1000,8501.1,O75387-1,559.0,...,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb,100757617
7674,r2532,Major Facilitator (Mfs) Tcdb:2.A.1.44.1,asn_L_e <=> asn_L_c,8501.1,"Transport, extracellular",-1000,1000,8501.1,O75387-1,559.0,...,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb,100757617
7675,r2534,Major Facilitator (Mfs) Tcdb:2.A.1.44.1,thr_L_e <=> thr_L_c,8501.1,"Transport, extracellular",-1000,1000,8501.1,O75387-1,559.0,...,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb,100757617


In [24]:
import gspread

# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns and Metabolites sheet.
rxns_sheet = cho_recon.worksheet('Rxns')

# We can extract the data using the get_all_records method and create pd DataFrames

# Reactions IDs, names, formulas, GPRs
rxns = pd.DataFrame(rxns_sheet.get_all_records())
del rxns[rxns.columns[0]]
rxns

,Curated,Reaction,Reaction Name,Reaction Formula,Subsystem,GPR_hef,GPR_fou,GPR_yeo,GPR_Recon3D,GPR_final,GPR_Final_Thanasis,Conf. Score,Curation Notes,References
0,PD,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal",10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
1,PD,10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial",10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,,1,No information available in the literature abo...,
2,PD,10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal",10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
3,PD,10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial",10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,,1,No information available in the literature abo...,
4,PD,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal",10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8193,,r2537,Utilized transport,lnlncgcoa_c <=> lnlncgcoa_r,Transport,,,,,,,,,
8194,,r2538,Utilized transport,dlnlcgcoa_c <=> dlnlcgcoa_r,Transport,,,,,,,,,
8195,,r2539,Postulated transport reaction,L2aadp6sa_c + L2aadp_m <=> L2aadp6sa_m + L2aadp_c,Transport,,,,,,,,,
8196,PD,ALLTTtm,"Allantoate transport via diffusion, mitochondria",alltt_c <=> alltt_m,"Transport, mitochondria",,,,,,,1,The transport of Allantoate from the cytoplasm...,


In [26]:
cho_rxns_list = list(rxns['Reaction'])
recon3D_rxns_list = list(recon3d_cho['m_reaction'])
recon3D_rxns_list

['11DOCRTSLte',
 '11DOCRTSTRNte',
 '12DHCHOLabc',
 '12DHCHOLt',
 '12DHCHOLt2',
 '12HTACRhr',
 '13DAMPPOX',
 '13DAMPPte',
 '14HMDZALThr',
 '14HMDZhr',
 '15KPROSTGF2c',
 '17AHPRGNLONEte',
 '17AHPRGSTRNte',
 '1A25DHVITD3te',
 '1HIBUP_SGLUhep',
 '1HMDGLUChr',
 '1OHMDZhr',
 '21HPRGNLONE',
 '24_25VITD2Hm',
 '24_25VITD3Hm',
 '25HVITD3c',
 '25HVITD3tin',
 '25VITD2Hm',
 '25VITD3Hm',
 '2AMACHYD',
 '2AMACSULT',
 '2AMADPTm',
 '2H3MVc',
 '2HATVACIDGLUChr',
 '2HATVACIDGLUCteb',
 '2HATVACIDOXDhc',
 '2HATVACIDhc',
 '2HATVACIDteb',
 '2HATVACIDthc',
 '2HATVLACGLUChr',
 '2HATVLACGLUCteb',
 '2HATVLACOXDhc',
 '2HATVLACteb',
 '2HATVLACthc',
 '2HBO',
 '2HBt2',
 '2HCO3_NAt',
 '2HIBUP_SGLUhep',
 '2HIVc',
 '2HYDOGOAT3t',
 '2HYOXPLCte',
 '2M3OVCOAm',
 '2MB2COAc',
 '2OBUTt',
 '2OXOADOXm',
 '2OXOADPTm',
 '34DHOXMANDte',
 '34DHOXPEGOX',
 '34DHPEAR',
 '34DHPHAMT',
 '34DHPHAte',
 '34DHPHELAT1tc',
 '34DHPHEt',
 '34DHPLACOX',
 '34DHPLACOX_NADP_',
 '34DHXMANDACOX',
 '34DHXMANDACOX_NADP_',
 '34HPPOR',
 '35DHPVShc',
 '35D

In [27]:
rxns_in_cho = []
rxns_not_in_cho = []

for rxn in recon3D_rxns_list:
    if rxn in cho_rxns_list:
        rxns_in_cho.append(rxn)
    elif rxn not in cho_rxns_list:
        rxns_not_in_cho.append(rxn)
              

In [30]:
rxns_in_cho

['13DAMPPOX',
 '15KPROSTGF2c',
 '24_25VITD2Hm',
 '24_25VITD3Hm',
 '25HVITD3c',
 '25VITD2Hm',
 '25VITD3Hm',
 '2AMACHYD',
 '2AMACSULT',
 '2AMADPTm',
 '2HBO',
 '2HBt2',
 '2HCO3_NAt',
 '2HYOXPLCte',
 '2MB2COAc',
 '2OXOADOXm',
 '2OXOADPTm',
 '34DHOXMANDte',
 '34DHOXPEGOX',
 '34DHPEAR',
 '34DHPHAMT',
 '34DHPHELAT1tc',
 '34DHPHEt',
 '34DHPLACOX',
 '34DHXMANDACOX',
 '34HPPOR',
 '3AIBTm',
 '3DSPHR',
 '3HAO',
 '3HBCDm',
 '3HBCOARc',
 '3HCO3_NAt',
 '3HKYNAKGAT',
 '3HLYTCL',
 '3HXKYNDCL',
 '3HXKYNOXDA',
 '3M4HDXPAC',
 '3MOX4HOXPGALDOX',
 '3MOXTYROX',
 '3NTD7l',
 '3SALAASPm',
 '3SALACBOXL',
 '3SALATAi',
 '3SALATAim',
 '41R1H2MAE12BOOX',
 '41R2A1H12BOOX',
 '42A12BOOX',
 '4HGLSDm',
 '4HPROLTASCT1',
 '4NPHSULT',
 '4OHPROIMINOtc',
 '5ADTSTSTERONEGLCte',
 '5ADTSTSTERONESULT',
 '5FTHFt2',
 '5HLTDL',
 '5HOXINDACTO2OX',
 '5HOXINDACTOXm',
 '5HOXINOXDA',
 '5HTRPDOX',
 '5HTRPVESSEC',
 '5HXKYNDCL',
 '5HXKYNOXDA',
 '5MTHFt',
 '5MTHFt2',
 'A4GALTc',
 'A4GALTg',
 'A4GNT1g',
 'A4GNT2g',
 'AACOAT',
 'AACTOOR',
 'AA

In [29]:
rxns_not_in_cho

['11DOCRTSLte',
 '11DOCRTSTRNte',
 '12DHCHOLabc',
 '12DHCHOLt',
 '12DHCHOLt2',
 '12HTACRhr',
 '13DAMPPte',
 '14HMDZALThr',
 '14HMDZhr',
 '17AHPRGNLONEte',
 '17AHPRGSTRNte',
 '1A25DHVITD3te',
 '1HIBUP_SGLUhep',
 '1HMDGLUChr',
 '1OHMDZhr',
 '21HPRGNLONE',
 '25HVITD3tin',
 '2H3MVc',
 '2HATVACIDGLUChr',
 '2HATVACIDGLUCteb',
 '2HATVACIDOXDhc',
 '2HATVACIDhc',
 '2HATVACIDteb',
 '2HATVACIDthc',
 '2HATVLACGLUChr',
 '2HATVLACGLUCteb',
 '2HATVLACOXDhc',
 '2HATVLACteb',
 '2HATVLACthc',
 '2HIBUP_SGLUhep',
 '2HIVc',
 '2HYDOGOAT3t',
 '2M3OVCOAm',
 '2OBUTt',
 '34DHPHAte',
 '34DHPLACOX_NADP_',
 '34DHXMANDACOX_NADP_',
 '35DHPVShc',
 '35DHPVSthc',
 '35DSMVhep',
 '3AIBtmi',
 '3DHCAS',
 '3DHCDCAS',
 '3DHCDCHOLabc',
 '3DHCDCHOLt',
 '3DHCDCHOLt2',
 '3DHCHOLabc',
 '3DHCHOLt',
 '3DHCHOLt2',
 '3DHDCAS',
 '3DHDCAabc',
 '3DHDCAt',
 '3DHDCAt2',
 '3DHLCAS',
 '3DHLCAabc',
 '3DHLCAt',
 '3DHLCAt2',
 '3HADICOAx',
 '3HIBUP_SGLUhep',
 '3HLVSTAChep',
 '3HPCOAHYD',
 '3HPVSTETCOAhcm',
 '3HPVSTETCOAhcx',
 '3HPVSTETteb',
 '3

In [19]:
len(recon3D_rxns_list)

8198

In [32]:
subset_df = recon3d_cho[recon3d_cho['m_reaction'].isin(rxns_not_in_cho)]
subset_df

,m_reaction,Reaction Name,m_metabolites,m_gene_reaction_rule,m_subsystem,Lower bound,Upper bound,m_gene,seq_uniprot,seq_len,...,seq_num_experimental_structures,struct_id,struct_chain_id,struct_chain_seq_coverage,struct_is_experimental,struct_pdb,struct_resolution,struct_chemicals,struct_file,CHO GPR
0,11DOCRTSLte,"Transport of 11-Deoxycortisol, Extracellular",11docrtsl_c + atp_c + h2o_c --> 11docrtsl_e + ...,5243.1,"Transport, extracellular",0,1000,5243.1,P08183-1,1280.0,...,21.0,3g61-A,A,80.5,1.0,3g61,4.35,2J8,3g61-A_clean.pdb,100682536
1,11DOCRTSTRNte,"Transport of 11-Deoxycorticosterone, Extracell...",11docrtstrn_c + atp_c + h2o_c --> 11docrtstrn_...,5243.1,"Transport, extracellular",0,1000,5243.1,P08183-1,1280.0,...,21.0,3g61-A,A,80.5,1.0,3g61,4.35,2J8,3g61-A_clean.pdb,100682536
2,12DHCHOLabc,12-Dehydrocholic acid ABC bile acid transporter,12dhchol_c + atp_c + h2o_c --> 12dhchol_e + ad...,8714.3 or 8647.1 or 8714.2 or 8714.1 or 1244.1...,"Transport, extracellular",0,1000,1244.1,Q92887-1,1545.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100767558 or 100762480 or 100767558 or 1007675...
3,12DHCHOLt,12-Dehydrocholic acid transport via bicarbonat...,12dhchol_e + hco3_c <=> 12dhchol_c + hco3_e,28234.1 or 10599.1 or 123264.1 or 200931.1,"Transport, extracellular",-1000,1000,123264.1,Q9NPD5-1,702.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100774298 or 100774298 or 100751085 or 100753765
4,12DHCHOLt2,12-Dehydrocholic acid transport via sodium cot...,12dhchol_e + 2.0 na1_e --> 12dhchol_c + 2.0 na1_c,6554.1 or 6555.1,"Transport, extracellular",0,1000,6555.1,Q12908-1,348.0,...,0.0,H19894-X,X,99.7,0.0,NaN,NaN,NaN,NP_000443.1_model1_clean-X_clean.pdb,100756998 or 100689396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7636,r2436,Mitochondrial Carrier (Mc) Tcdb:2.A.29.8.3,crn_m + odecrn_c <=> crn_c + odecrn_m,788.1,"Transport, mitochondrial",-1000,1000,788.1,O43772-1,301.0,...,0.0,H05589-X,X,100.0,0.0,NaN,NaN,NaN,NP_000378.1_model1_clean-X_clean.pdb,100765000
7637,r2437,Mitochondrial Carrier (Mc) Tcdb:2.A.29.8.3,crn_m + pcrn_c <=> crn_c + pcrn_m,788.1,"Transport, mitochondrial",-1000,1000,788.1,O43772-1,301.0,...,0.0,H05589-X,X,100.0,0.0,NaN,NaN,NaN,NP_000378.1_model1_clean-X_clean.pdb,100765000
7638,r2438,Mitochondrial Carrier (Mc) Tcdb:2.A.29.8.3,c4crn_c + crn_m <=> c4crn_m + crn_c,788.1,"Transport, mitochondrial",-1000,1000,788.1,O43772-1,301.0,...,0.0,H05589-X,X,100.0,0.0,NaN,NaN,NaN,NP_000378.1_model1_clean-X_clean.pdb,100765000
7639,r2439,"Zinc (Zn2+)-Iron (Fe2+) Permease (Zip), Tcdb:2...",hco3_c + so4_e --> hco3_e + so4_c,10861.1,"Transport, extracellular",0,1000,10861.1,Q9H2B4-1,701.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103159861
